In [3]:
%matplotlib inline

# plotting
import matplotlib as mpl
mpl.style.use('ggplot')
import matplotlib.pyplot as plt

# math and data manipulation
import numpy as np
import pandas as pd

# set random seeds 
from numpy.random import seed
from dateutil.parser import parse
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor

# modeling
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Bidirectional, GRU, Flatten, Dropout, TimeDistributed
# progress bar
from tqdm import tqdm

In [4]:
train = pd.read_csv('../data/CV/train_new.csv',parse_dates=['timestamp'],index_col=0)
train.head(5)

,series_id,timestamp,consumption,temperature
496608,100003,2017-10-24 00:00:00,187969.954324,8.75
496609,100003,2017-10-24 01:00:00,191001.727781,8.70
496610,100003,2017-10-24 02:00:00,187969.954324,8.20
496611,100003,2017-10-24 03:00:00,187969.954324,7.75
496612,100003,2017-10-24 04:00:00,191001.727781,7.70


In [5]:
test = pd.read_csv('../data/CV/test_train_new.csv',index_col=0,parse_dates=['timestamp'])
test.head(5)

,series_id,timestamp,consumption,temperature
496944,100003,2017-11-07 00:00:00,191001.727781,5.35
496945,100003,2017-11-07 01:00:00,191001.727781,5.25
496946,100003,2017-11-07 02:00:00,191001.727781,5.20
496947,100003,2017-11-07 03:00:00,194033.501238,5.30
496948,100003,2017-11-07 04:00:00,191001.727781,5.35


In [6]:
my_submission = pd.read_csv('../data/CV/test_new.csv')
my_submission.head(3)

,pred_id,series_id,timestamp,temperature,consumption,prediction_window
0,0,100003,2017-11-16 00:00:00,NaN,0,hourly
1,1,100003,2017-11-16 01:00:00,NaN,0,hourly
2,2,100003,2017-11-16 02:00:00,NaN,0,hourly


In [7]:
def create_lagged_features(df, lag=1):
    if not type(df) == pd.DataFrame:
        df = pd.DataFrame(df, columns=['consumption'])
    
    def _rename_lag(ser, j):
        ser.name = ser.name + f'_{j}'
        return ser
        
    # add a column lagged by `i` steps
    for i in range(1, lag + 1):
        df = df.join(df.consumption.shift(i).pipe(_rename_lag, i))

    df.dropna(inplace=True)
    return df

In [8]:
def prepare_training_data(consumption_series, lag):
    """ Converts a series of consumption data into a
        lagged, scaled sample.
    """
    # scale training data
    scaler = MinMaxScaler(feature_range=(0, 1))
    consumption_vals = scaler.fit_transform(consumption_series.values.reshape(-1, 1))
    
    # convert consumption series to lagged features
    consumption_lagged = create_lagged_features(consumption_vals, lag=lag)
        
    cols = list(consumption_lagged.columns)
    
    for i in range(len(consumption_lagged.columns)-1):
        consumption_lagged[cols[i]] = consumption_lagged[cols[i]] - consumption_lagged[cols[i+1]]
        
    cols.remove('consumption')
    # X, y format taking the first column (original time series) to be the y
    X = consumption_lagged.drop('consumption', axis=1).values
    y = consumption_lagged.consumption.values
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], X.shape[1])
    #X = X.reshape(X.shape[0], X.shape[1])
    
    return X, y, scaler

In [9]:
lag = 24

In [10]:
clf = ExtraTreesRegressor(n_estimators=100) #HuberRegressor()
X_main = []
y_main = []
norm_models = {}

In [11]:
for ser_id, ser_data in train.groupby('series_id'):
    X,y,scaler = prepare_training_data(ser_data.consumption, lag)
    norm_models[ser_id] = scaler
    X_main += list(X)
    y_main += list(y)
for ser_id, ser_data in test.groupby('series_id'):
    X,y,scaler = prepare_training_data(ser_data.consumption, lag)
    norm_models[ser_id] = scaler
    X_main += list(X)
    y_main += list(y)    

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_main, y_main, test_size = .2)

In [23]:
clf = GradientBoostingRegressor(n_estimators=100)

In [24]:
clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [25]:
pred = clf.predict(X_test)

In [26]:
def mae(y_pred,y_true):
    l = []
    for i in range(len(y_pred)):
        if y_true[i] != 0:
            l.append(np.abs(y_pred[i] - y_true[i])*100/y_true[i])
    return np.mean(l)

In [27]:
print (mae(pred,y_test))

-58.2539573964


In [17]:
def generate_hourly_forecast(num_pred_hours, consumption, model, scaler, lag):
    """ Uses last hour's prediction to generate next for num_pred_hours, 
        initialized by most recent cold start prediction. Inverts scale of 
        predictions before return.
    """
    # allocate prediction frame
    preds_scaled = np.zeros(num_pred_hours)
    
    # initial X is last lag values from the cold start
    X = scaler.transform(consumption.values.reshape(-1, 1))[-lag:]
    X = X[::-1]
    latest = X[0]
    for i in range(len(X)-1):
        X[i] = X[i] - X[i+1]
        
    # forecast
    for i in range(num_pred_hours):
        # predict scaled value for next time step
        X = X.reshape((1,lag))
        yhat = model.predict(X)[0]
        preds_scaled[i] = yhat+latest
        
        # update X to be latest data plus prediction
        X = pd.Series(X.ravel()).shift(1).fillna(yhat).values

    # revert scale back to original range
    hourly_preds = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).ravel()
    return hourly_preds

In [28]:
pred_window_to_num_preds = {'hourly': 24, 'daily': 7, 'weekly': 2}
pred_window_to_num_pred_hours = {'hourly': 24, 'daily': 7 * 24, 'weekly': 2 * 7 * 24}
model = clf
num_test_series = my_submission.series_id.nunique()


for ser_id, pred_df in my_submission.groupby('series_id'):
        
    num_pred_hours = pred_df.shape[0]
    
    # prepare cold start data
    series_data = test[test.series_id == ser_id].consumption
    cold_X, cold_y, scaler = prepare_training_data(series_data, lag)
    
    # make hourly forecasts for duration of pred window
    preds = generate_hourly_forecast(num_pred_hours, series_data, model, scaler, lag)
    
    # reduce by taking sum over each sub window in pred window
    # reduced_preds = [pred.sum() for pred in np.split(preds, num_preds)]
    
    # store result in submission DataFrame
    ser_id_mask = my_submission.series_id == ser_id
    my_submission.loc[ser_id_mask, 'consumption'] = preds

In [29]:
my_submission.head(3)

,pred_id,series_id,timestamp,temperature,consumption,prediction_window
0,0,100003,2017-11-16 00:00:00,NaN,184281.289200,hourly
1,1,100003,2017-11-16 01:00:00,NaN,184052.492301,hourly
2,2,100003,2017-11-16 02:00:00,NaN,184066.336796,hourly


In [30]:
actual = pd.read_csv('../data/CV/test_actual.csv')
actual.head(3)

,actual
0,200097.048151
1,200097.048151
2,200097.048151


In [31]:
np.sqrt(mean_squared_error(my_submission.consumption,actual.actual))

143619.73987556767

In [32]:
mae(my_submission.consumption,actual.actual)

57.603090351049573

In [130]:
pd.Series([1,2,3]).shift(1)

0    NaN
1    1.0
2    2.0
dtype: float64